In [144]:
import logging
import pathlib
import sys
from typing import List
import random
import os
import time
from subprocess import check_output
import scipy.sparse as sparse

import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize

In [145]:
mallet_path = pathlib.Path("/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/mallet-2.0.8/bin/mallet")

def infer_thetas(path_model, num_topics, docs):
    num_iterations = 1000
    doc_topic_thr = 0.0
    holdout_corpus = path_model / "infer_data" / "corpus.txt"
    with holdout_corpus.open("w", encoding="utf8") as fout:
        for i, t in docs:
            fout.write(f"{i} 0 {t}\n")
    print(f"-- -- Mallet corpus.txt for inference created.")

    # Get inferencer
    inferencer = path_model / "model_data" / "inferencer.mallet"

    # Files to be generated thoruogh Mallet
    corpus_mallet_inf = path_model / "infer_data" / "corpus_inf.mallet"
    doc_topics_file = path_model / "infer_data" / "doc-topics-inf.txt"


    # Extract pipe
    # Get corpus file
    path_corpus = path_model / "train_data" / "corpus.mallet"
    if not path_corpus.is_file():
        print(f"-- Pipe extraction: Could not locate corpus file")

    # Create auxiliary file with only first line from the original corpus file
    path_txt = path_model / "train_data" / "corpus.txt"
    with path_txt.open('r', encoding='utf8') as f:
        first_line = f.readline()
    path_aux = path_model / "train_data" / "corpus_aux.txt"
    with path_aux.open('w', encoding='utf8') as fout:
        fout.write(first_line + '\n')

    # We perform the import with the only goal to keep a small file containing the pipe
    print(f"-- Extracting pipeline")
    path_pipe = path_model / "train_data" / "import.pipe"

    cmd = mallet_path.as_posix() + \
        ' import-file --use-pipe-from %s --input %s --output %s'
    cmd = cmd % (path_corpus, path_aux, path_pipe)

    try:
        print(f'-- Running command {cmd}')
        check_output(args=cmd, shell=True)
    except:
        print('-- Failed to extract pipeline. Revise command')

    # Import data to mallet
    print('-- Inference: Mallet Data Import')

    #
    cmd = mallet_path.as_posix() + \
        ' import-file --use-pipe-from %s --input %s --output %s'
    cmd = cmd % (path_pipe, holdout_corpus, corpus_mallet_inf)

    try:
        print(f'-- Running command {cmd}')
        check_output(args=cmd, shell=True)
    except Exception as e:
        print(e)
        print('-- Mallet failed to import data. Revise command')

    # Get topic proportions
    print('-- Inference: Inferring Topic Proportions')

    cmd = mallet_path.as_posix() + \
        ' infer-topics --inferencer %s --input %s --output-doc-topics %s ' + \
        ' --doc-topics-threshold ' + str(doc_topic_thr) + \
        ' --num-iterations ' + str(num_iterations)
    cmd = cmd % (inferencer, corpus_mallet_inf, doc_topics_file)

    try:
        print(f'-- Running command {cmd}')
        check_output(args=cmd, shell=True)
    except:
        print('-- Mallet inference failed. Revise command')

    cols = [k for k in np.arange(2, num_topics + 2)]
    thetas32 = np.loadtxt(doc_topics_file, delimiter='\t', dtype=np.float32, usecols=cols)
    thetas32[thetas32 < 3e-3] = 0
    thetas32 = normalize(thetas32, axis=1, norm='l1')
    thetas32 = sparse.csr_matrix(thetas32, copy=True)
    
    path_save = path_model / "infer_data" / "thetas.npz"
    sparse.save_npz(path_save, thetas32)
    
    return thetas32.shape
    

In [146]:
################
# Paths to data
################
path_parquets = pathlib.Path("/export/usuarios_ml4ds/cggamella/NP-Search-Tool/sample_data/all_processed")
path_place_without_lote = path_parquets / "minors_insiders_outsiders_origen_sin_lot_info.parquet"
path_place_esp = path_parquets / "df_esp_langid.parquet"
path_manual_stops = "/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/stopwords_sin_duplicados"
path_eq = "/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/eq.txt"

In [147]:
################
# Read data
################
print(f"-- -- Reading data from {path_place_esp} and {path_place_without_lote}")
processed = pd.read_parquet(path_place_esp)
cols = processed.columns.values.tolist()
print(f"-- -- Data read from {path_place_esp}: {len(processed)} rows.")
# set identifier as column so we dont loose it
processed['identifier'] = processed.index
print(f"-- -- Columns: {cols}")
place_without_lote = pd.read_parquet(path_place_without_lote)
print(f"-- -- Data read from {path_place_without_lote}: {len(place_without_lote)} rows.")

#########################
# Get additional metadata
#########################
# Merge 'processed' with 'place_without_lote' to get info about the source of the tender (minors, insiders, outsiders)
processed = pd.merge(processed, place_without_lote, how='left', on='id_tm')
processed.set_index('identifier', inplace=True)  # Preserved index
processed = processed[cols + ["origen"]]  #  Keep necessary columns
print(f"-- -- Data merged: {len(processed)} rows.")
#print(f"-- -- Sample: {processed.head()}")

-- -- Reading data from /export/usuarios_ml4ds/cggamella/NP-Search-Tool/sample_data/all_processed/df_esp_langid.parquet and /export/usuarios_ml4ds/cggamella/NP-Search-Tool/sample_data/all_processed/minors_insiders_outsiders_origen_sin_lot_info.parquet
-- -- Data read from /export/usuarios_ml4ds/cggamella/NP-Search-Tool/sample_data/all_processed/df_esp_langid.parquet: 2618584 rows.
-- -- Columns: ['id_tm', 'raw_text', 'lemmas', 'lang']
-- -- Data read from /export/usuarios_ml4ds/cggamella/NP-Search-Tool/sample_data/all_processed/minors_insiders_outsiders_origen_sin_lot_info.parquet: 3110261 rows.
-- -- Data merged: 2618584 rows.


In [148]:
#####################
# Filter stops /eqs #
#####################
# Filter stops
stopwords = set()
# Lista para registrar los nombres de los archivos procesados
archivos_procesados = []
# Iterar sobre cada archivo en el directorio especificado
for archivo in os.listdir(path_manual_stops):
    if archivo.endswith('.txt'):
        ruta_completa = os.path.join(path_manual_stops, archivo)
        with open(ruta_completa, 'r', encoding='utf-8') as f:
            stopwords.update(f.read().splitlines())
        # Registrar el archivo procesado
        archivos_procesados.append(archivo)
print(f"-- -- There are {len(stopwords)} stopwords")
def eliminar_stopwords(fila):
    return ' '.join([palabra for palabra in fila.split() if palabra not in stopwords])
start = time.time()
processed['lemmas'] = processed['lemmas'].apply(eliminar_stopwords)
print(f"-- -- Stops filtered in {time.time() - start}")

# Filter eqs
start = time.time()
pares_diccionario = {}
compiled_regexes = {}
with open(path_eq, 'r') as archivo:
    for linea in archivo:
        linea = linea.strip()
        palabras = linea.split(':')
        if len(palabras) < 2:
            print(f"Línea omitida o incompleta: '{linea}'")
            continue
        pares_diccionario[palabras[0]] = palabras[1]
pares_diccionario = \
    dict(sorted(pares_diccionario.items(), key=lambda x: x[0]))
print(f"-- -- There are {len(pares_diccionario)} equivalences")
print("-- -- Eq dict constructed in :", time.time() - start)

def replace_keywords(lst, keyword_dict):
    return " ".join([keyword_dict.get(word, word) for word in lst])

start = time.time()
processed["lemmas_split"] = processed['lemmas'].apply(lambda x: x.split())
processed['lemmas'] = processed['lemmas_split'].apply(
    lambda x: replace_keywords(x, pares_diccionario))
processed = processed.drop(columns=['lemmas_split'])
print("-- -- Eq substituted in:", time.time() - start)

-- -- There are 111425 stopwords
-- -- Stops filtered in 5.834235429763794
-- -- There are 2827 equivalences
-- -- Eq dict constructed in : 0.015024423599243164
-- -- Eq substituted in: 8.673483848571777


In [149]:
pares_diccionario["nanociència_i"]

'nanociencia'

In [150]:
############################
# Filter by lemmas min len #
############################
min_lemmas = 1
min_lemmas_tm = 2
processed['len'] = processed['lemmas'].apply(lambda x: len(x.split()))
#processed = processed[processed['len'] >= min_lemmas]

In [151]:
all = processed.copy()
minors = all[all.origen == "minors"]
outsiders = all[all.origen == "outsiders"]
insiders = all[all.origen == "insiders"]

In [152]:
# Get tenders that were not included in the modeling process
all_not_tm = all[(all['len'] <= min_lemmas_tm) & (all['len'] >= min_lemmas)]
all_not_tm

,id_tm,raw_text,lemmas,lang,origen,len
identifier,,,,,,
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/495,4,Obras de reparación del Centro de Información ...,rellano,es,minors,1
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/491,8,Actuaciones de mejora de las condiciones de se...,condición_confort garcía_aguilar,es,minors,2
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/490,9,Suministro De Armarios Para La Unidad De Bolsa...,religiosas_sagrados_corazones,es,minors,1
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/489,10,Soterramiento de contenedores de residuos en P...,soterramiento_contenedor duración,es,minors,2
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/487,12,Suministro De Lencería Para Los Hospitales De ...,lencería llàtzer,es,minors,2
...,...,...,...,...,...,...
PlataformasAgregadasSinMenores_202401.zip/PlataformasAgregadasSinMenores.atom/7,3110253,Servicios para el desarrollo y gestión del pro...,euskara_merkataritzara pueblo,es,outsiders,2
PlataformasAgregadasSinMenores_202401.zip/PlataformasAgregadasSinMenores.atom/5,3110255,Contrato de servicios para la prestación del s...,socorrismo_piscina,es,outsiders,1
PlataformasAgregadasSinMenores_202401.zip/PlataformasAgregadasSinMenores.atom/3,3110257,Servicios y suministros para la seguridad inte...,veiasa,es,outsiders,1


In [153]:
all[all['len'] > min_lemmas_tm]

,id_tm,raw_text,lemmas,lang,origen,len
identifier,,,,,,
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/499,0,Reforma de elementos de ventilación exterior d...,villa_molina caudal confort,es,minors,3
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/498,1,Servicios de calibrado y certificado de dos de...,calibrado detector trasvase_reversible,es,minors,3
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/494,5,Diversos materiales para el acondicionamiento ...,rotonda arriat decoración jardinera maceta via...,es,minors,6
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/493,6,Adquisición de un espectrómetro de masas con d...,espectrómetro_masa aguas_confederación hidrogr...,es,minors,3
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/480,19,Suministro de un SWITCH para el Laboratorio de...,switch aguas_confederación hidrográfica_ebro,es,minors,3
...,...,...,...,...,...,...
PlataformasAgregadasSinMenores_202401.zip/PlataformasAgregadasSinMenores.atom/18,3110242,Suministro de un cabezal de repuesto para la s...,cabezal soplante_edar cursos medios,es,outsiders,4
PlataformasAgregadasSinMenores_202401.zip/PlataformasAgregadasSinMenores.atom/17,3110243,Prestación de un servicio de consultoría y asi...,cualitativo brecha_digital condición_prescripc...,es,outsiders,12
PlataformasAgregadasSinMenores_202401.zip/PlataformasAgregadasSinMenores.atom/6,3110254,Suministro y montaje de VALORIZA: Separación y...,valoriza separación reciclado agroalimentario,es,outsiders,4


In [154]:
minors_not_tm = minors[(minors['len'] <= min_lemmas_tm) & (minors['len'] >= min_lemmas)]
minors_not_tm

,id_tm,raw_text,lemmas,lang,origen,len
identifier,,,,,,
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/495,4,Obras de reparación del Centro de Información ...,rellano,es,minors,1
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/491,8,Actuaciones de mejora de las condiciones de se...,condición_confort garcía_aguilar,es,minors,2
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/490,9,Suministro De Armarios Para La Unidad De Bolsa...,religiosas_sagrados_corazones,es,minors,1
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/489,10,Soterramiento de contenedores de residuos en P...,soterramiento_contenedor duración,es,minors,2
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/487,12,Suministro De Lencería Para Los Hospitales De ...,lencería llàtzer,es,minors,2
...,...,...,...,...,...,...
contratosMenoresPerfilesContratantes_202401.zip/contratosMenoresPerfilesContratantes.atom/5,2159022,Servicios de telecomunicaciones e intenet Manc...,intenet,es,minors,1
contratosMenoresPerfilesContratantes_202401.zip/contratosMenoresPerfilesContratantes.atom/3,2159024,Proyecto ejecucion instalacion fibra optica ed...,fibra_optico aljucen,es,minors,2
contratosMenoresPerfilesContratantes_202401.zip/contratosMenoresPerfilesContratantes.atom/2,2159025,Adquisición de frigorifico para la sala de téc...,frigorífico icasst,es,minors,2


In [155]:
minors[minors['len'] > min_lemmas_tm]

,id_tm,raw_text,lemmas,lang,origen,len
identifier,,,,,,
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/499,0,Reforma de elementos de ventilación exterior d...,villa_molina caudal confort,es,minors,3
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/498,1,Servicios de calibrado y certificado de dos de...,calibrado detector trasvase_reversible,es,minors,3
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/494,5,Diversos materiales para el acondicionamiento ...,rotonda arriat decoración jardinera maceta via...,es,minors,6
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/493,6,Adquisición de un espectrómetro de masas con d...,espectrómetro_masa aguas_confederación hidrogr...,es,minors,3
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/480,19,Suministro de un SWITCH para el Laboratorio de...,switch aguas_confederación hidrográfica_ebro,es,minors,3
...,...,...,...,...,...,...
contratosMenoresPerfilesContratantes_202401.zip/contratosMenoresPerfilesContratantes.atom/48,2158979,Campaña de Navidad Degusta Jaén: miel gourmet ...,degusta miel gourmet,es,minors,3
contratosMenoresPerfilesContratantes_202401.zip/contratosMenoresPerfilesContratantes.atom/42,2158985,Trabajos de albañilería para empotrar canaliza...,empotrar canalización casería,es,minors,3
contratosMenoresPerfilesContratantes_202401.zip/contratosMenoresPerfilesContratantes.atom/36,2158991,"Servicio azafatas, control de accesos y gastos...",azafatas taquilla femau,es,minors,3


In [156]:
outsiders_not_tm = outsiders[(outsiders['len'] <= min_lemmas_tm) & (outsiders['len'] >= min_lemmas)]
outsiders_not_tm

,id_tm,raw_text,lemmas,lang,origen,len
identifier,,,,,,
PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137_1.atom/451,2816132,Refuerzo de firme en la VP 4013 Melgar de Arri...,vp santervás,es,outsiders,2
PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137_1.atom/448,2816133,Desarrollo del programa de intervención socioe...,intervención_socioeducativo psicosocial,es,outsiders,2
PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137_1.atom/442,2816137,Servicios De Mediación De Seguros,mediación_seguro,es,outsiders,1
PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137_1.atom/440,2816138,Panel de indicadores de Desarrollo Sostenible ...,urdaibai,es,outsiders,1
PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137_1.atom/436,2816139,Servicio de Celaduría y Labores Complementaria...,celaduría labores_complementarias,es,outsiders,2
...,...,...,...,...,...,...
PlataformasAgregadasSinMenores_202401.zip/PlataformasAgregadasSinMenores.atom/7,3110253,Servicios para el desarrollo y gestión del pro...,euskara_merkataritzara pueblo,es,outsiders,2
PlataformasAgregadasSinMenores_202401.zip/PlataformasAgregadasSinMenores.atom/5,3110255,Contrato de servicios para la prestación del s...,socorrismo_piscina,es,outsiders,1
PlataformasAgregadasSinMenores_202401.zip/PlataformasAgregadasSinMenores.atom/3,3110257,Servicios y suministros para la seguridad inte...,veiasa,es,outsiders,1


In [157]:
outsiders[outsiders['len'] > min_lemmas_tm]

,id_tm,raw_text,lemmas,lang,origen,len
identifier,,,,,,
PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137_1.atom/453,2816130,Refuerzo de Firme en la VP 3001 Renedo de Esgu...,vp_renedo esgueva_duero villabañez pk,es,outsiders,4
PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137_1.atom/452,2816131,Refuerzo de Firme en la VP 6603 Mota del Marqu...,vp mota_marqués lp arión,es,outsiders,4
PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137_1.atom/444,2816136,Servicios de arquitecto para la remodelación d...,hogei urte euskalduna_jauregia palacio_euskalduna,es,outsiders,4
PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137_1.atom/435,2816140,Adquisición de material (44.000 determinacione...,gamma_interferón bovino ensayo_inmunoenzimático,es,outsiders,3
PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137_1.atom/434,2816141,Adquisición de 31.000 determinaciones para el ...,post_mortem elisa_encefalopatía espongiformes_...,es,outsiders,6
...,...,...,...,...,...,...
PlataformasAgregadasSinMenores_202401.zip/PlataformasAgregadasSinMenores.atom/18,3110242,Suministro de un cabezal de repuesto para la s...,cabezal soplante_edar cursos medios,es,outsiders,4
PlataformasAgregadasSinMenores_202401.zip/PlataformasAgregadasSinMenores.atom/17,3110243,Prestación de un servicio de consultoría y asi...,cualitativo brecha_digital condición_prescripc...,es,outsiders,12
PlataformasAgregadasSinMenores_202401.zip/PlataformasAgregadasSinMenores.atom/6,3110254,Suministro y montaje de VALORIZA: Separación y...,valoriza separación reciclado agroalimentario,es,outsiders,4


In [158]:
insiders_not_tm = insiders[(insiders['len'] <= min_lemmas_tm) & (insiders['len'] >= min_lemmas)]
insiders_not_tm

,id_tm,raw_text,lemmas,lang,origen,len
identifier,,,,,,
licitacionesPerfilesContratanteCompleto3_2018.zip/licitacionesPerfilesContratanteCompleto3_20200522_234632_1.atom/494,2159028,Contratación de la Obra de creación de un nuev...,rambla_segadors,es,insiders,1
licitacionesPerfilesContratanteCompleto3_2018.zip/licitacionesPerfilesContratanteCompleto3_20200522_234632_1.atom/486,2159031,Prestación del servicio de vigilancia y limpie...,pabellón_cubierto explotación_cafetería,es,insiders,2
licitacionesPerfilesContratanteCompleto3_2018.zip/licitacionesPerfilesContratanteCompleto3_20200522_234632_1.atom/481,2159032,Servicios de mantenimiento y desarrollo de los...,ministerio_defensa,es,insiders,1
licitacionesPerfilesContratanteCompleto3_2018.zip/licitacionesPerfilesContratanteCompleto3_20200522_234632_1.atom/480,2159033,Reforma planta de hospitalización de Medicina ...,planta_hospitalización medicina_interna,es,insiders,2
licitacionesPerfilesContratanteCompleto3_2018.zip/licitacionesPerfilesContratanteCompleto3_20200522_234632_1.atom/474,2159036,Servicio de reconocimientos médicos de vigilan...,reconocimiento_médico,es,insiders,1
...,...,...,...,...,...,...
licitacionesPerfilesContratanteCompleto3_202401.zip/licitacionesPerfilesContratanteCompleto3.atom/6,2816123,Obras de construcción de una nueva Instalación...,instalación_deportiva elemental,es,insiders,2
licitacionesPerfilesContratanteCompleto3_202401.zip/licitacionesPerfilesContratanteCompleto3.atom/5,2816124,Contrato de suministro e instalación de equipa...,acogido_recuperación europea_nextgenerationeu,es,insiders,2
licitacionesPerfilesContratanteCompleto3_202401.zip/licitacionesPerfilesContratanteCompleto3.atom/3,2816126,Contratación del servicio de asistencia sanita...,primario_población llanos_aridane,es,insiders,2


In [159]:
insiders[insiders['len'] > min_lemmas_tm]

,id_tm,raw_text,lemmas,lang,origen,len
identifier,,,,,,
licitacionesPerfilesContratanteCompleto3_2018.zip/licitacionesPerfilesContratanteCompleto3_20200522_234632_1.atom/488,2159030,Red separativa de alcantarillado en el barrio ...,separativo_alcantarillado palombera riestre na...,es,insiders,4
licitacionesPerfilesContratanteCompleto3_2018.zip/licitacionesPerfilesContratanteCompleto3_20200522_234632_1.atom/477,2159035,Abastecimiento y depuración del pueblo de Soto...,depuración pueblo hermandad_campoo,es,insiders,3
licitacionesPerfilesContratanteCompleto3_2018.zip/licitacionesPerfilesContratanteCompleto3_20200522_234632_1.atom/447,2159042,"Contratación del suministro, personalización y...",personalización almacenaje promocional_merchan...,es,insiders,4
licitacionesPerfilesContratanteCompleto3_2018.zip/licitacionesPerfilesContratanteCompleto3_20200522_234632_1.atom/426,2159049,Primer contrato derivado del Acuerdo Marco par...,prótesi externo_órtesi unidad_ortoprótesi hosp...,es,insiders,4
licitacionesPerfilesContratanteCompleto3_2018.zip/licitacionesPerfilesContratanteCompleto3_20200522_234632_1.atom/424,2159051,Primer contrato derivado del Acuerdo Marco par...,prótesi externo_órtesi unidad_ortoprótesi hosp...,es,insiders,4
...,...,...,...,...,...,...
licitacionesPerfilesContratanteCompleto3_202401.zip/licitacionesPerfilesContratanteCompleto3.atom/15,2816114,Obra de impermeabilización de la cubierta de R...,impermeabilización_cubierta rectorado_campus upv,es,insiders,3
licitacionesPerfilesContratanteCompleto3_202401.zip/licitacionesPerfilesContratanteCompleto3.atom/13,2816116,"Contratación de las obras de montaje, decoraci...",caseta_feria emasesa pascual_márquez,es,insiders,3
licitacionesPerfilesContratanteCompleto3_202401.zip/licitacionesPerfilesContratanteCompleto3.atom/11,2816118,Servicio de Evaluación Externa de la Calidad d...,evaluación_externa unigreen europeas eacea com...,es,insiders,5


## ALL

In [160]:
path_model = pathlib.Path("/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/Mallet/es_Mallet_all_55_topics_FINAL")
docs = all_not_tm[["id_tm", "lemmas"]].values
num_topics = 55

In [161]:
thetas = sparse.load_npz((path_model / "model_data/TMmodel" / "thetas.npz"))
thetas.toarray().shape

(792502, 55)

In [162]:
infer_thetas(path_model, num_topics, docs)

-- -- Mallet corpus.txt for inference created.
-- Extracting pipeline
-- Running command /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/mallet-2.0.8/bin/mallet import-file --use-pipe-from /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/Mallet/es_Mallet_all_55_topics_FINAL/train_data/corpus.mallet --input /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/Mallet/es_Mallet_all_55_topics_FINAL/train_data/corpus_aux.txt --output /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/Mallet/es_Mallet_all_55_topics_FINAL/train_data/import.pipe
-- Inference: Mallet Data Import
-- Running command /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/mallet-2.0.8/bin/mallet import-file --use-pipe-from /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/Mallet/es_Mallet_all_55_topics_FINAL/train_data/i

(1443707, 55)

## OUTSIDERS

In [163]:
path_model = pathlib.Path("/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/Mallet/es_Mallet_outsiders_30_topics_FINAL")
docs = outsiders_not_tm[["id_tm", "lemmas"]].values
num_topics = 30

In [164]:
infer_thetas(path_model, num_topics, docs)

-- -- Mallet corpus.txt for inference created.
-- Extracting pipeline
-- Running command /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/mallet-2.0.8/bin/mallet import-file --use-pipe-from /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/Mallet/es_Mallet_outsiders_30_topics_FINAL/train_data/corpus.mallet --input /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/Mallet/es_Mallet_outsiders_30_topics_FINAL/train_data/corpus_aux.txt --output /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/Mallet/es_Mallet_outsiders_30_topics_FINAL/train_data/import.pipe
-- Inference: Mallet Data Import
-- Running command /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/mallet-2.0.8/bin/mallet import-file --use-pipe-from /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/Mallet/es_Mallet_outsiders_30_t

(72962, 30)

## INSIDERS

In [165]:
path_model = pathlib.Path("/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/Mallet/es_Mallet_insiders_12_topics_FINAL")
docs = insiders_not_tm[["id_tm", "lemmas"]].values
num_topics = 12

In [166]:
infer_thetas(path_model, num_topics, docs)

-- -- Mallet corpus.txt for inference created.
-- Extracting pipeline
-- Running command /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/mallet-2.0.8/bin/mallet import-file --use-pipe-from /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/Mallet/es_Mallet_insiders_12_topics_FINAL/train_data/corpus.mallet --input /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/Mallet/es_Mallet_insiders_12_topics_FINAL/train_data/corpus_aux.txt --output /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/Mallet/es_Mallet_insiders_12_topics_FINAL/train_data/import.pipe
-- Inference: Mallet Data Import
-- Running command /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/mallet-2.0.8/bin/mallet import-file --use-pipe-from /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/Mallet/es_Mallet_insiders_12_topic

(318474, 12)

## MINORS

In [167]:
path_model = pathlib.Path("/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/Mallet/es_Mallet_minors_40_topics_FINAL")
docs = minors_not_tm[["id_tm", "lemmas"]].values
num_topics = 40

In [168]:
infer_thetas(path_model, num_topics, docs)

-- -- Mallet corpus.txt for inference created.
-- Extracting pipeline
-- Running command /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/mallet-2.0.8/bin/mallet import-file --use-pipe-from /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/Mallet/es_Mallet_minors_40_topics_FINAL/train_data/corpus.mallet --input /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/Mallet/es_Mallet_minors_40_topics_FINAL/train_data/corpus_aux.txt --output /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/Mallet/es_Mallet_minors_40_topics_FINAL/train_data/import.pipe
-- Inference: Mallet Data Import
-- Running command /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/mallet-2.0.8/bin/mallet import-file --use-pipe-from /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/models/Mallet/es_Mallet_minors_40_topics_FINAL/

(1052271, 40)